In [1]:
import pandas as pd
import os
import gc
import pickle
import numpy as np

from time import sleep

import gspread_dataframe as gs_df

os.chdir("c:/Users/rsilva/Google Drive/projects/programming/automation")

# INITIALIZE DRIVER
from selenium import webdriver
from sapiens import LoginPage, KEYS, Page_sapiens
import functions
#import sistemas


% reload_ext autoreload
% autoreload 2

CPF = '33831640807'
PASS_1 = "Fer465023@ur_"
PASS_2 = "Fer465023"

### Loga Driver no Sapiens

In [3]:
def start_sapiens(cpf, senha, sapies=None):   
   
    page = functions.init_browser(webdriver.Firefox(), 'rsilva', 'Savorthemom3nts')
    driver = LoginPage(page.driver).login(cpf, senha)
    
    if sapiens:
        
        sapiens.reset_driver(driver)
        
    else:
        
        sapiens = Page_sapiens(driver, {})
        
    return sapiens

### Cria/Carrega o Data Frame para exportar os dados

In [2]:
#database = pd.read_csv("registros_cadin.csv")

In [3]:
#database = database.dropna(subset=["CPF"])

In [5]:
#registros = {row[1]["CPF"]:{k:v for k,v in row[1].items()} for row in database.iterrows()}

In [5]:
sapiens = start_sapiens(CPF, PASS_2, None)

UnboundLocalError: local variable 'sapiens' referenced before assignment

### Autenticar e Abrir planilhas Google

In [5]:
#gc = functions.authenticate_google("files/anatel.json")
#sht = gc.open(title='RA')

In [ ]:
#registros = sht.worksheet('Registros')
#registros = sht.get_worksheet(0)
#df_registros = gs_df.get_as_dataframe(registros, dtype=str)
#registros_rf = sht.worksheet('Registros_RF')
#df_registro_rf = gs_df.get_as_dataframe(registros_rf, dtype=str)
df = pd.DataFrame(columns=KEYS)

In [7]:
#processo = sht.worksheet("53504.005667/2018-21")
#df_processo = gs_df.get_as_dataframe(processo, dtype=str)

path = "files/"
devolvidos = pd.read_excel(path+"devolvidos.xlsx", dtype=str)

In [8]:
devolvidos.head()


,Entidade,UF,Fistel,Seq.,Receita,Serviço,Data Vencimento,Nº Comunicado,Data Comunicado,Valor Comunicado,Gestor,Situação
0,11500 EMPREENDIMENTOS LTDA - EPP,SP,50402126742,24,1329,28,2015-03-31 00:00:00,010-066406/2017,2017-11-09 07:29:01,296.5,sandroramos,Débito
1,24/7 INTELIGENCIA DIGITAL LTDA.,SP,50406459290,18,1329,19,2015-03-31 00:00:00,010-066408/2017,2017-11-09 07:29:10,80.85,sandroramos,Débito
2,A C A SANTOS-ME,SP,50406300933,18,1329,19,2017-03-31 00:00:00,010-066412/2017,2017-11-09 07:29:14,246.16,sandroramos,Débito
3,A. JARDIM COMPETIÇOES LTDA,SP,50402287401,22,1329,28,2015-03-31 00:00:00,010-066414/2017,2017-11-09 07:29:15,53.91,sandroramos,Débito
4,A. R. - SOUZA SERVICOS AGRICOLAS LTDA - ME,SP,50407984232,14,1329,19,2017-03-31 00:00:00,010-066418/2017,2017-11-09 07:29:19,279.73,sandroramos,Débito


In [9]:
lista_cpf[741]

'24886230830'

In [14]:
counter = 0

for cpf in lista_cpf[741:]:
    
    cpf = str(cpf)    
        
    if len(cpf) in (12, 13, 14):
        
        next
        
        while len(cpf) < 14:
            
            cpf = '0' + str(cpf)
            
    elif len(cpf) <= 11:
    
        while len(cpf) < 11:

            cpf = '0' + str(cpf)
            
    else:
        
        raise ValueError("CPF/CNPJ inválido {}".format(cpf))

    registro = sapiens.get_registro(cpf)    
   
    if not registro:
    
        sapiens.pesquisa_dados(cpf)
            
        registro = sapiens.get_registro(cpf)
        
        counter += 1
        
   
    if not (df["CPF"] == registro.get(cpf)).any():
        
        df = df.append(pd.Series(registro), ignore_index=True)
        
    if counter > 100:
        
        df.to_csv("registro_sapiens_devolvidos.csv", index=False)
        
        page = functions.init_browser(webdriver.Firefox(), 'rsilva', 'Savorthemom3nts')
        
        driver = LoginPage(page.driver).login('33831640807', 'Fer465023@ur_')        
               
        sapiens.reset_driver(driver)
        
        sleep(10)        
                
        counter = 0          

TimeoutException: Message: 


In [45]:
df = df.drop_duplicates()
df = df.fillna("")

In [46]:
df["Endereço_Completo"] = df.apply(lambda row: " ".join([str(row["Logradouro"]), 
                                                         str(row["Número"]), 
                                                         str(row["Complemento"]),
                                                         str(row["Bairro"]), "\n", "CEP:", 
                                                         str(row["Cep"]), 
                                                         str(row["Cidade"])]
                                                       ), axis=1
                                  )

In [47]:
#df = df.sort_values(by="Nome")
df.shape

(24, 17)

In [65]:
#writer = pd.ExcelWriter("files/planilha_obito.xlsx")
#df.to_excel(writer, "consulta_RF", index=False, encoding='utf-8')
#writer.save()

In [129]:
#gs_df.set_with_dataframe(registros_rf, df, row=1, include_column_header=True)
#gs_df.set_with_dataframe(registros, pd.DataFrame(df["Nome"]), row=1, col=9, include_column_header=False)
#gs_df.set_with_dataframe(registros, pd.DataFrame(df["Endereço_Completo"]), row=1, col=12, include_column_header=False)

In [48]:
gs_df.set_with_dataframe(registros, df, row=743, col=14, include_column_header=False)
#gs_df.set_with_dataframe(processo, df, row=1, col=15, include_column_header=True)

In [40]:
lista_fistel = df_registros["FISTEL"]

,CPF,Nome,Mãe,Data de Nascimento,Sexo,Ano do Óbito,Nacionalidade,Título de Eleitor,Situação Cadastral,Fone,Logradouro,Número,Complemento,Bairro,Cidade,Cep,Endereço_Completo
0,280.866.158-47,ALEXANDRE RIBEIRO COUTINHO,OLINDA DA SILVA COUTINHO,06-05-1971,MASCULINO,,BRASILEIRO,0171875770108,REGULAR,02033342,RUA FRANCISCO ANTONIO MIRANDA,88,,EMERLINO MATARAZZO,SAO PAULO/SP,03809-130,RUA FRANCISCO ANTONIO MIRANDA 88 EMERLINO MAT...
1,976.282.398-20,CARLOS BARBOSA DOS SANTOS,MARIA DE PAULA BARBOSA,29-05-1959,MASCULINO,,BRASILEIRO,0019969790116,REGULAR,81677191,RUA MANOEL DOS SANTOS,321,,PONTILHAO,CRUZEIRO/SP,12704-200,RUA MANOEL DOS SANTOS 321 PONTILHAO \n CEP: 1...
2,096.175.138-03,CELSO PEREIRA,MARIA ROSA PEREIRA,10-11-1968,MASCULINO,,BRASILEIRO,0163050660108,REGULAR,00000000,RUA LUIZ MELGES,25,,CENTRO,POMPEIA/SP,17580-000,RUA LUIZ MELGES 25 CENTRO \n CEP: 17580-000 P...
3,270.032.178-21,FABIO RIBEIRO VAZ LOBO FILHO,MARIA APARECIDA DE OLIVEIRA VAZ LOBO,16-07-1973,MASCULINO,,BRASILEIRO,0206397630116,REGULAR,92156103,RUA COSTA RICA,58,APARTAMENTO 2,JARDIM LAVINIA,MOCOCA/SP,13736-350,RUA COSTA RICA 58 APARTAMENTO 2 JARDIM LAVINIA...
4,317.494.448-10,FABIO ROBERTO DA MOTTA,LEONILDA DA MOTTA,02-07-1981,MASCULINO,,BRASILEIRO,0248329210191,REGULAR,33415590,RUA PASCOAL MUSSOLINE,190,,JARDIM AEROPORTO,CANDIDO MOTA/SP,19880-000,RUA PASCOAL MUSSOLINE 190 JARDIM AEROPORTO \n...


In [13]:
import gc

gc.collect()

2114